In [53]:
from utils import *
import pandas as pd
import re
import numpy as np
import os
import subprocess

In [2]:
# Bills
INDIR_115_HR = './data/115/bills/hr'
INDIR_115_S = './data/115/bills/s'

# Concurrent Resolutions
INDIR_115_HCONRES = './data/115/bills/hconres'
INDIR_115_SCONRES = './data/115/bills/sconres'

# Joint Resolutions
INDIR_115_HJRES = './data/115/bills/hjres'
INDIR_115_SJRES = './data/115/bills/sjres'

# Simple Resolutions
INDIR_115_HRES = './data/115/bills/hres'
INDIR_115_SRES = './data/115/bills/sres'

INDIR_ALL = [INDIR_115_HR, INDIR_115_S, 
              INDIR_115_HCONRES, INDIR_115_SCONRES, 
              INDIR_115_HJRES, INDIR_115_SJRES, 
              INDIR_115_HRES, INDIR_115_SRES
             ]

In [3]:
data = []

for i in INDIR_ALL:
    print('Processing {}'.format(i))
    for d in walk_dirs(i):
        data.append(d)
        
df_115 = pd.DataFrame(data)

Processing ./data/115/bills/hr
Processing ./data/115/bills/s
Processing ./data/115/bills/hconres
Processing ./data/115/bills/sconres
Processing ./data/115/bills/hjres
Processing ./data/115/bills/sjres
Processing ./data/115/bills/hres
Processing ./data/115/bills/sres


In [4]:
df_115.head()

,Congress,Directory,ID,Number,Subtype,Summary,Type,Version
0,115,./data/115/bills/hr/hr1206/text-versions/ih,115_HR1206_IH,HR1206,HR,1,Bills,IH
1,115,./data/115/bills/hr/hr4265/text-versions/ih,115_HR4265_IH,HR4265,HR,0,Bills,IH
2,115,./data/115/bills/hr/hr689/text-versions/ih,115_HR689_IH,HR689,HR,1,Bills,IH
3,115,./data/115/bills/hr/hr689/text-versions/rfs,115_HR689_RFS,HR689,HR,1,Bills,RFS
4,115,./data/115/bills/hr/hr689/text-versions/eh,115_HR689_EH,HR689,HR,1,Bills,EH


In [5]:
df_115.Type.value_counts()

Bills                     7867
Simple Resolutions        1104
Joint Resolutions          246
Concurrent Resolutions     173
Name: Type, dtype: int64

In [6]:
len(df_115)

9390

# Split train, val, test

In [7]:
# df_115_valid.groupby(['Number', 'Version']).size()

In [9]:
df_115_valid = df_115[(df_115.Summary==1) 
                      & (df_115.Version != 'N/A')
                      & (df_115.Version != 'EAS')
                      & (df_115.Version != 'EAH')
                      & (df_115.Version != 'EAS2') 
                      & (df_115.Version != 'PAP')
    ]

df_select = df_115_valid.groupby(['Congress', 'Number']).first()

In [10]:
df_select.head()

Directory  \
Congress Number                                                         
115      HCONRES1   ./data/115/bills/hconres/hconres1/text-version...   
         HCONRES10  ./data/115/bills/hconres/hconres10/text-versio...   
         HCONRES11  ./data/115/bills/hconres/hconres11/text-versio...   
         HCONRES12  ./data/115/bills/hconres/hconres12/text-versio...   
         HCONRES13  ./data/115/bills/hconres/hconres13/text-versio...   

                                  ID  Subtype  Summary  \
Congress Number                                          
115      HCONRES1   115_HCONRES1_RDS  HCONRES        1   
         HCONRES10  115_HCONRES10_IH  HCONRES        1   
         HCONRES11  115_HCONRES11_IH  HCONRES        1   
         HCONRES12  115_HCONRES12_IH  HCONRES        1   
         HCONRES13  115_HCONRES13_IH  HCONRES        1   

                                      Type Version  
Congress Number                                     
115      HCONRES1   Concurrent Resolutions     RDS  
         HCONRES10  Concurrent Resolutions      IH  
         HCONRES11  Concurrent Resolutions      IH  
         HCONRES12  Concurrent Resolutions      IH  
         HCONRES13  Concurrent Resolutions      IH

In [11]:
len(df_select)

5192

In [17]:
train, validate, test = np.split(df_select.sample(frac=1), [int(.8*len(df_select)), int(.9*len(df_select))])

print(len(train), len(validate), len(test))

4153 519 520


In [22]:
train.to_csv('./data/train.txt')
validate.to_csv('./data/validate.txt')
test.to_csv('./data/test.txt')

In [28]:
df_train = pd.read_csv('./data/train.txt')

In [29]:
df_train.head()

,Congress,Number,Directory,ID,Subtype,Summary,Type,Version
0,115,SRES290,./data/115/bills/sres/sres290/text-versions/ats,115_SRES290_ATS,SRES,1,Simple Resolutions,ATS
1,115,HR2522,./data/115/bills/hr/hr2522/text-versions/ih,115_HR2522_IH,HR,1,Bills,IH
2,115,HR280,./data/115/bills/hr/hr280/text-versions/ih,115_HR280_IH,HR,1,Bills,IH
3,115,HR944,./data/115/bills/hr/hr944/text-versions/ih,115_HR944_IH,HR,1,Bills,IH
4,115,HR3141,./data/115/bills/hr/hr3141/text-versions/ih,115_HR3141_IH,HR,1,Bills,IH


# Create tmp Process

In [18]:
def clean_bill(text):
    try:
        split = re.split(r'(?i)\sAN\sACT\s{3,}|\sA\sBIL[L]*\s{2,}|\sA*\s[A-Z]*\sRESOLUTION\s{3,}', text)
        if len(split) > 1: 
            text1 = split[1]
        else:
            text1 = text.split('In the House of Representatives, U. S.,')[1]
        text21 = re.split(r'\s{2,}Passed the (Senate |House of Representatives )', text1)[0] # remove words after 'Attest:'
        text22 = re.split(r'\s{2,}Attest:\s{2,}', text21)[0] # remove words after 'Attest:'
        text23 = re.split(r'\s{2,}(Union |House )*Calendar No\.\s\d+\s{2,}', text22)[0] # 
        text24 = re.split(r'\s{2,}Speaker of the House of Representatives\.\s{2,}', text23)[0]
        text3 = re.sub(r'[ ][\n][ ]+', r' ', text24) # somtimes one sentence may be break into multiple lines
        text4 = re.sub(r'\<all\>\s*$', r'', text3) # remove <all> in the end
        text5 = re.sub(r'\s', r' ', text4) # replace all kinds of white space (\n\t ) with ' ' 
        text6 = text5.replace('--', ' -- ') # add axtra space around --
        text7 = re.sub(r',\s+SECTION 1.', r'. SECTION 1', text6) # replace ',' before section 1 with '.'
        text8 = re.sub(r'(?i)(\sSEC)(\.)(\s\d)(\s*\.)', r'\1TION\3 ', text7) # replace 'SEC. x.' with 'SECTION x '
#         text9 = re.sub(r'Calendar No[\d\w\s\.]+$', r'', text8) # remove extra Calendar No.... in the end
        text10 = text8.replace(';', '.')
        text11 = text10.replace("''. ", " ")
        return text11.strip()
    except:
        print('!!! Error at cleaning text:')
        print(text)
#         return ' '
        raise

In [29]:
def get_clean_summary(row):
    path = row['Directory']
    with open('/'.join(path.split('/')[0:6])+'/data.xml') as f:
        dict1 = xmltodict.parse(f.read())
    summary_text = dict1['bill']['summary']['#text']
    text6 = [i.strip().rstrip() for i in summary_text.split('\n')]
    text7 = [re.sub('\(Sec\. [0-9]+\)', '', i) for i in text6]
    text8 = [i for i in text7 if i!='']
    summary = ' '.join(text8)
    return summary

In [60]:
TMPDIR = './tmp/out3'

def split_file(row):
    indir_b = os.path.join(row['Directory'], 'document.txt')
    filename_b = 'BILL'+'_'+row['ID']
    tmp_b = os.path.join(TMPDIR, filename_b)
    out_b = os.path.join(TMPDIR, filename_b+'.out')
    
    indir_s = os.path.join(row['Directory'], '..', '..', 'data.json')
    filename_s = 'SUMMARY'+'_'+row['ID']
    tmp_s = os.path.join(TMPDIR, filename_s)
    out_s = os.path.join(TMPDIR, filename_s+'.out')
    
    with open(indir_b, 'r') as f:
        text_b = f.read()
        
    with open(tmp_b, 'w') as f:
        f.write(clean_bill(text_b))
    
    command = ['java', "-classpath", "/home/lucy/stanford-corenlp/stanford-corenlp-full-2016-10-31/stanford-corenlp-3.7.0.jar:/home/lucy/stanford-corenlp/stanford-corenlp-full-2016-10-31/stanford-corenlp-3.7.0-models.jar", 'edu.stanford.nlp.process.DocumentPreprocessor', tmp_b]
    with open(out_b, "w") as outfile:
        subprocess.run(command, stdout=outfile)

        
    with open(indir_s, 'r') as f:
        text_s = json.load(f)['summary']['text']
    
    with open(tmp_s, 'w') as f:
        f.write(text_s)
    
    command = ['java', "-classpath", "/home/lucy/stanford-corenlp/stanford-corenlp-full-2016-10-31/stanford-corenlp-3.7.0.jar:/home/lucy/stanford-corenlp/stanford-corenlp-full-2016-10-31/stanford-corenlp-3.7.0-models.jar", 'edu.stanford.nlp.process.DocumentPreprocessor', tmp_s]

    with open(out_s, "w") as outfile:
        subprocess.run(command, stdout=outfile)


    
    os.remove(tmp_b)
    os.remove(tmp_s)
        
#     return tmp_b
    

In [309]:
OUTDIR = './tmp/bill'
# TMPDIR_SUMMARY = './tmp/summaries/'
# OUTDIR_BILL = './out/bills/'
# OUTDIR_SUMMARY = './out/summaries/'

In [62]:
%%time
df_115_valid.apply(split_file, axis=1)

CPU times: user 26.4 s, sys: 25.7 s, total: 52.1 s
Wall time: 30min 57s


0       None
2       None
3       None
4       None
5       None
8       None
9       None
10      None
11      None
14      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
30      None
31      None
32      None
33      None
35      None
36      None
37      None
        ... 
9356    None
9357    None
9358    None
9359    None
9360    None
9361    None
9362    None
9363    None
9364    None
9365    None
9366    None
9368    None
9370    None
9371    None
9372    None
9373    None
9374    None
9376    None
9377    None
9378    None
9379    None
9380    None
9381    None
9382    None
9383    None
9384    None
9385    None
9387    None
9388    None
9389    None
Length: 6775, dtype: object